# Type Hints

Python Version: 3.5 but with many improvements added in the following releases.

[Official Docs](https://docs.python.org/3/library/typing.html)

[Mypy cheatsheet](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)

Python supports type hints which say what type of value a variable or function parameter is. They are not enforced by the Python interpreter but can be used by static type checkers like Mypy or Pylance in VS Code to catch errors.

## Why use type hints?

Type hints catch errors and make code more readable. For instance, a function signature like `def convert(date)` can be confusing on its own; is the `date` argument a string, a datetime object, or something else? Type hints concisely show us how a program expects to be used.

Type hints are at their best catching unexpected return values from functions. For instance, we might have a hole in our logic that results in a function returning `None` (the value returned when we do not explicitly `return` anything) and then later consume that return value but not account for it being `None`. Another way they're immensely helpful is that type checkers _know the types and return values of everything in the standard library_ and most things in imported modules. They check that our understanding of others' code is correct.

When I started using type checking, one of the most common warnings I saw on our Django website was a lot of code which looks like `db.objects().last()` where I assumed I had an object but in fact this method can return `None` if no objects have been created in the database yet. Type hints know the expect return values of these `db` methods and catch such errors.


```py
import requests

from models import InstagramOAuthToken

def get_instagram_post():
    # objects.last() returns None if there are no objects in the DB!
    OAuthToken = InstagramOAuthToken.objects.last()
    # I need an if statement here to check if the token is None
    token = OAuthToken.token
    response = requests.get(f"https://graph.instagram.com/me/media?fields=id,caption,media_url&access_token={token}")
```


Another common error in my code: the iterator returned by pymarc's `MARCReader` does not always return a `Record` object, sometimes it returns `None` if a record is malformed (we can see this by reading an empty text file):

```python
from pymarc import MARCReader

with open('example.mrc', 'rb') as fh:
    reader = MARCReader(fh)
    for record in reader:
        # record could be None so we need this if condition
        if record:
            dosomething(record)
```



## How do they work?

Type hints for variables and parameters consistent of a colon `:` followed by the type. Type hints for functions' return values are indicated by an arrow `->`. Types are the same class names we see when we call `type()` on a value, such as `int`, `str`, `list`, `dict`, etc.

In [5]:
from random import randint

# basic type hints
name: str = "Eric"
age: int = 87

# Python "knows" randint returns an int so this works
def random_age() -> int:
    return randint(0, 100)

age = random_age()

def greeting(fname: str) -> str:
    return f"Hello, {fname}!"

print(greeting(age)) # warning! age is an int, not a str

Hello, 1!


The above is an excellent example because the `f` string returned by `greeting` actually doesn't care that it has an `int` inside it, it coerces the `int` to a string. So there's no error thrown, but our function wasn't used in the intended way! The type hints are catching something that is technically valid code, which does not raise an exception, but is not what we intended.

## Lists & Dicts

When we annotate something as a list, we can also specify what types of values are in the list. For instance, `List[int]` is a list of integers.

When declaring type hints for dicts, we declare the type of the dict's keys and then its values inside brackets with a comma in between. For instance, `Dict[str, int]` is a dictionary where the keys are strings and the values are integers.



In [2]:
english_numbers: dict[int, str] = {
    1: "one",
    2: "two",
    3: "three",
}
number_words: list[str] = list(english_numbers.values())

What if our type isn't one of the core ones included in the global namespace? We can import classes from modules to use in our type hints. There is also a `typing` module with many important utilities.

In [3]:
from os import stat, stat_result # stat_result is a specific object structure
from pathlib import Path # we are only using Path in our type hint

def stat_tmp_doc(doc: Path) -> stat_result:
    # The "/" operator doesn't work without a Path argument, can't use with 2 strings
    doc = "/tmp" / doc
    return stat(doc)

stat_tmp_doc(Path("records.mrc"))

os.stat_result(st_mode=33204, st_ino=2413209, st_dev=16777234, st_nlink=1, st_uid=502, st_gid=20, st_size=972986, st_atime=1710467870, st_mtime=1685738676, st_ctime=1701375669)

## Union types

But what if our variable takes on multiple types? It's common for functions that search for some pattern or datum to either return data of a predictable type or, if nothing is found, `None`. We can model this in our type hints with a `Union` type. `Union` can be imported from the `typing` module or, in Python 3.10 and later, we can use the built-in `|` operator.

In [60]:
from random import choices
import re
import string
from typing import Union

text: str = ''.join(choices(string.ascii_letters, k=20))
# re.search returns either a Match object or None if the pattern isn't found
match: Union[re.Match, None] = re.search(r"a", text)
# Python 3.10+
# match: re.Match|None = re.search(r"a", text)
print(text, match)

ZziMCEgfahKczJStzAVA <re.Match object; span=(8, 9), match='a'>


There's no limit to the number of unions we can have, though at some point it becomes a code "smell" that our logic is too complex.

In [99]:
from random import choices
import re

# function returns none if no matches, matched string if only 1, or number of matches if there are multiple
def get_letter_a(text: str) -> str|int|None:
    matches: None|list[str] = re.findall(r"a", text)
    if len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        return len(matches)
    else:
        return None

print(get_letter_a(''.join(choices('abcdefgh', k=5))))

a


`typing` also has an `Any` type which might be useful in situations like this or when we are unsure what potential types an external library might return.

In [100]:
from random import choice
from typing import Any

protean: Any = choice([1, 'a', choice, [], {}, None])
print(type(protean))

<class 'str'>


Finally, `typing` has an `Optional` type which means "the given type or `None`". So `Optional[int]` is the same as `Union[int, None]`. In practice, typing `int|None` is probably quicker and clearer.

In [79]:
import re
from typing import Optional

match: Optional[re.Match] = re.search(r"xyz", "abc")
print(match)

None


## Working Around Errors

But isn't using `Any` the same as not using type hints at all? True, but this explicitly tells us that the variable's type is unpredictable. If we are working with badly behaved code, whether our own or from external libraries, `Any` annotations can help us work any type errors (perhaps temporarily, until code is improved) while still applying type hints consistently.

Most type checkers also provide a means to ignore particular lines of code so we can keep working without being distracted by known errors we're not able to address at the moment. Sometimes the types are in fact _wrong_ for some third-party code. For instance, I wrote some typed code using the `pysftp` module (last updated in 2016) which seems confused about the type of the `log` parameter (it wants it to be boolean but in fact works with an integer or `None`):

```python
import pysftp

# debug is either 1 (int) or None but pysftp thinks it's a boolean
with pysftp.Connection(
    host="sftp.example.com",
    log=debug, # type: ignore
    port=22
) as sftp:
    sftp.put("/tmp/text", "/home")
```

## Literal Values

If we know all the specific values a variable can take on, we can use the `Literal` type from the `typing` module. My actual `pyfstp` code defines `debug: Literal[1, None]` and those values come from arguments passed on the command line.

There are many scenarios where a specific set of strings provides even more stringent type checking than a `str` type hint.

In [101]:
from datetime import date
from typing import Literal

def season(d: date) -> Literal["winter", "spring", "summer", "fall"]:
    if d.month in [12, 1, 2]:
        return "winter"
    elif d.month in [3, 4, 5]:
        return "spring"
    elif d.month in [6, 7, 8]:
        return "summer"
    # implies month is in [9, 10, 11]
    return "fall"

print(season(date.today()))

spring


## Your Own Classes

Have a custom structure which our code uses over and over? Type hints give us a concrete reason to define a class for it. There was a discussion a while back in the python4lib Slack channel about when to start using object-oriented programming and folks felt more comfortable writing functions. That makes perfect sense, but type hints can provide an incentive to create a little more structure and abstract commons patterns from our data.

In [ ]:
from datetime import date

class Student():
    def __init__(self, name: str, grad_date: str|None) -> None:
        self.name: str = name
        self.graduation_date: date | None = date.fromisoformat(grad_date) if grad_date else None

    @property
    def is_alumni(self) -> bool:
        # can't do this grad date math if it's a string
        return self.graduation_date is not None and self.graduation_date < date.today()

person = {
    "name": "Eric",
    "graduation_date": "2022-05-15"
}

# this raises an error because student is a dict, not a Student object with the
# is_alumni property calculated from their graduation date
if person.is_alumni:
    print(f"ask {person.name} for a donation")

person = Student(**person)

if person.is_alumni:
    print(f"ask {person.name} for a donation") # this works!